In [2]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("../alerts.db")
cursor = conn.cursor()

# Fetch incidents from the past 7 days
query = """
SELECT type, severity, location, status
FROM alerts
WHERE timestamp >= DATE('now', '-7 days')
ORDER BY timestamp DESC;
"""

cursor.execute(query)
incidents = cursor.fetchall()  # Fetch all rows

# Close the database connection
conn.close()


In [3]:
print(incidents)

[('fire', 'critical', 'Al Mushrif, Abu Dhabi', 'in progress'), ('assault', 'high', 'Al Raha Beach, Abu Dhabi', 'resolved'), ('assault', 'critical', 'Reem Island, Abu Dhabi', 'resolved'), ('fire', 'warning', 'Yas Mall, Abu Dhabi', 'in progress'), ('assault', 'critical', 'Corniche Road, Abu Dhabi', 'unresolved'), ('fire', 'warning', 'Khalidiya Mall, Abu Dhabi', 'in progress'), ('fire', 'critical', 'Hamdan Street, Abu Dhabi', 'resolved')]


In [4]:
# Format incidents into a readable string
incident_text = "\n".join(
    f"Date: Type: {row[0]}, Severity: {row[1]}, Location: {row[2]}, Status: {row[3]}"
    for row in incidents
)

# Create the prompt for Gen AI
prompt = f"""
You are an AI assistant generating a weekly incident report. Be straight forward and maintain a structure that can be placed into a pdf file. Do not talk to me. Only give me the text that would be put in the PDF.
Analyze the following incident data and summarize key insights:

{incident_text}

Generate a structured report with:
- Total incidents
- Most common incident type
- Severity distribution
- Locations with the highest incidents
- Open vs closed case summary
- Key observations & recommendations
"""


In [5]:
incident_text

'Date: Type: fire, Severity: critical, Location: Al Mushrif, Abu Dhabi, Status: in progress\nDate: Type: assault, Severity: high, Location: Al Raha Beach, Abu Dhabi, Status: resolved\nDate: Type: assault, Severity: critical, Location: Reem Island, Abu Dhabi, Status: resolved\nDate: Type: fire, Severity: warning, Location: Yas Mall, Abu Dhabi, Status: in progress\nDate: Type: assault, Severity: critical, Location: Corniche Road, Abu Dhabi, Status: unresolved\nDate: Type: fire, Severity: warning, Location: Khalidiya Mall, Abu Dhabi, Status: in progress\nDate: Type: fire, Severity: critical, Location: Hamdan Street, Abu Dhabi, Status: resolved'

In [7]:
from http.client import HTTPException
import os
from openai import OpenAI
from dotenv import load_dotenv
import re

load_dotenv()
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

LLM_MODEL = "meta/llama-3.1-70b-instruct"
SUMMARIZER_MODEL = "deepseek-ai/deepseek-r1"

def extract_post_think_content(text: str) -> str:
    """Extract content after </think> tag."""
    match = re.search(r'</think>\s*(.*?)$', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text


embedding_client = OpenAI(api_key=NVIDIA_API_KEY, base_url="https://integrate.api.nvidia.com/v1")
llm_client = OpenAI(api_key=NVIDIA_API_KEY, base_url="https://integrate.api.nvidia.com/v1")
try:
        messages = [
            {"role": "system", "content": "You are an AI assistant generating a weekly incident report."},
            {"role": "user", "content": prompt}
        ]
        response = llm_client.chat.completions.create(
            model=SUMMARIZER_MODEL,
            messages=messages,
            temperature=0.6,
            top_p=0.7,
            max_tokens=4096,
            stream=False
        )
        summary = response.choices[0].message.content.strip()
        processed_summary = extract_post_think_content(summary)
except Exception as e:
    # logger.error(f"Error generating summary: {e}")
    print("error")
    # raise HTTPException(status_code=500, detail="Error generating summary.")

In [37]:
_summary


'<think>\nOkay, let\'s start by looking at the user\'s query. They want a weekly incident report based on the provided data. The instructions are to be straightforward and maintain a structure suitable for a PDF. No conversational text, just the report content.\n\nFirst, I need to parse the incident data. There are seven incidents listed with dates, types, severities, locations, and statuses. Let me count each category.\n\nTotal incidents: There are seven entries, so total is 7.\n\nMost common incident type: Fires and assaults each have entries. Let\'s count. Fire occurs on 2023-10-02, 2023-10-04, 2023-10-06, and 2023-10-07. Wait, no. Wait, looking at the data again:\n\n1. Fire (critical, Al Mushrif)\n2. Assault (high, Al Raha Beach)\n3. Assault (critical, Reem Island)\n4. Fire (warning, Yas Mall)\n5. Assault (critical, Corniche Road)\n6. Fire (warning, Khalidiya Mall)\n7. Fire (critical, Hamdan Street)\n\nSo Fire occurs 4 times (incidents 1,4,6,7) and Assault 3 times (2,3,5). So Fire 

In [38]:
from datetime import datetime, timedelta

print(datetime.now() - timedelta(days=7))

2025-01-28 11:43:41.167508


In [39]:
%pip install reportlab



Note: you may need to restart the kernel to use updated packages.


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors

def save_report_to_pdf(report_text, filename="weekly_incident_report.pdf"):
    """Generate a PDF file with a bordered report."""
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    c.setFont("Helvetica", 12)

    margin_x = 40
    margin_y = 50
    text_width = width - 2 * margin_x
    text_height = height - 2 * margin_y

    c.setStrokeColor(colors.black)
    c.setLineWidth(2)  
    c.rect(margin_x, margin_y, text_width, text_height)  

    # Write text inside the border
    y_position = height - margin_y - 20  # Start position for text
    line_height = 14  # Spacing between lines

    for line in report_text.split("\n"):
        c.drawString(margin_x + 10, y_position, line)  # Indent text inside border
        y_position -= line_height
        if y_position < margin_y + 20:  # Create new page if needed
            c.showPage()
            c.setFont("Helvetica", 12)
            y_position = height - margin_y - 20
            c.setStrokeColor(colors.black)
            c.setLineWidth(2)
            c.rect(margin_x, margin_y, text_width, text_height)

    c.save()
    print(f"Report saved as {filename}")

save_report_to_pdf(processed_summary)


Report saved as weekly_incident_report.pdf
